# Recommender with Spotify API Integration

## 1: Create Spotify Authorization Token

Due to Spotify API restrictions, you need to add SpotifyForDevelopers to your Spotify account to be able to run the code in this notebook. Since you can't "log in" per se from the notebook itself, creating this token allows you to validate your credentials and retrieve information from your Spotify account. 

### Steps

1. Go to https://developer.spotify.com/dashboard/ and log in (or sign up if you don't already have an account).  
2. Go to https://developer.spotify.com/console/post-playlists/ and press "Get Token".
3. Select the following checkboxes:  
    a. playlist-modify-public  
    b. playlist-modify-private  
    c. user-read-recently-played  
4. Press "Request Token" and "Agree". Now you should see a field value under **OAuth Token**. Copy this, as we'll need it later.  
5. Next, open Spotify. Open the drop-down next to your name in the top-right corner, and select **Account**.  
6. Copy the **Username** as we'll need it later.  
7. Open a terminal on your system. 
8. Enter the 2 following commands one by one in your terminal, replacing the <> with the variables you copied earlier. This basically allows you to validate your Spotify credentials without hard-coding it in this notebook's code cells. If you're on a Mac, replace the _SET_ keyword with _export_.  
**setx SPOTIFY_AUTHORIZATION_TOKEN \<yourOAuthToken\>  
setx SPOTIFY_USER_ID \<yourUsername\>**  

<img src="./terminal.JPG">

Restart your system for the changes to take effect. 

**Note: Spotify Access Tokens expire every hour, so you may need to go through the previous steps again if any of the code cells fail.**

In [1]:
#!pip install spotipy
import spotipy
import json
import os
import requests

# instantiate global variables
auth_token = os.environ.get("SPOTIFY_AUTHORIZATION_TOKEN") 
user_id = os.getenv("SPOTIFY_USER_ID")

# if the below lines returns None, please restart your system
print(auth_token)
print(user_id)

BQAk5yNsAbseA9jZAngj_tvO1rvXS1Of62ByU0Wbn3-Stx1-khEQqQSx6vCzX7R8PVmiZUZ9EIN2U83JzexaiQqxb1b467KsUKpu9pNFhs9PRTuyYubN2WElM9N2s7hS7sQvlHuS0oi9Gii23Kd7dKDs1Vm6OHzDBQIZC_D1VF3f6a9F9y1UI8r0jjM6M-rVHL3PmLGQ780qkVA0QR1W5hmmKJtEkcX4c5ZyTO1HwoeO0yTr3eAAYI8FbqQ0PCmfNYE9ug
31eul4dtufccxjoldxw6veem35mq


In [2]:
def getAPIrequest(auth_token, url):
    """
    Function to place GET requests to the Spotify API.
    """
    response = requests.get(
            url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {auth_token}"
            }
        )
    return response

def getLastPlayedSongs(numOfTracks):
    """
    Function to retrieve the last numOfTracks songs played by the user.
    """
    url = f"https://api.spotify.com/v1/me/player/recently-played?limit={numOfTracks}"
    response = getAPIrequest(auth_token, url)
    response_json = response.json()
    songs = []
    #print(json.dumps(response_json, indent=4))
    for song in response_json["items"]:
        songs.append(song)
    #print(songs)
    return songs


## 2: Retrieve User's Recently Played Songs
https://developer.spotify.com/console/get-recently-played/
 
JSON format is specified here. 

In [3]:
num = int(input("How many tracks would you like to visualize? "))
lastPlayed = getLastPlayedSongs(num)
print(f"\nHere are the last {num} tracks you listened to on Spotify:")
for index, track in enumerate(lastPlayed):
    print(f"\n {index+1}- {track}")

How many tracks would you like to visualize? 2

Here are the last 2 tracks you listened to on Spotify:

 1- {'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2XVJkKNBPA834yV4NKISBw'}, 'href': 'https://api.spotify.com/v1/artists/2XVJkKNBPA834yV4NKISBw', 'id': '2XVJkKNBPA834yV4NKISBw', 'name': 'Southside Trio', 'type': 'artist', 'uri': 'spotify:artist:2XVJkKNBPA834yV4NKISBw'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', '

## 3: Get User's Preferences

In [4]:
ref_tracks = input("\nEnter a list of up to 5 tracks to be used as seed tracks: ") # enter space separated number of the track
ref_tracks = ref_tracks.split()
seed_tracks = [lastPlayed[int(i)-1] for i in ref_tracks]
#print(seed_tracks)


Enter a list of up to 5 tracks to be used as seed tracks: 1


## 4: Convert to Model-friendly Input

In [6]:
def get_song_info(song_list):
    """
    Function to get the name and release year of seed tracks. 
    """
    song_dict = {}
    for item in range(len(song_list)):
        x = {song_list[item]['track']['album']['name']:song_list[item]['track']['album']['release_date'][:4]}
        song_dict.update(x)
    return song_dict

print(get_song_info(seed_tracks))

{'The Law of Attraction': '2021'}


## 5: Make Recommendations